Misspellings of curse words?  

When building vocab, consider punctuation. Tokenize it first? Right now it just splits on whitespace. Moses probably has something for this

In [1]:
from time import time

import numpy as np
from scipy.sparse import csr_matrix, vstack

np.random.seed(1234) # help reproducibility

In [2]:
# y == 0 if not offensive
# y == 1 if offensive
with open('data/OLIDv1.0/olid-training-v1.0.tsv') as f:
    raw = [x.strip().split('\t') for x in f.readlines()[1:]]
    x_raw = [r[1] for r in raw]
    y = np.array([0 if r[2] == 'NOT' else 1 for r in raw])
    
with open('data/jigsaw/train.csv') as f:
    pass
    # can't use .readlines() because that splits by '\n'. you need to read by character
    # raw = [x.strip().split(',') for x in f.readlines()[1:-1]]

In [15]:
off = 100 * (np.sum(y) / y.shape[0])
not_off = 100 * (1 - (np.sum(y) / y.shape[0]))
print('Offensive\tNot offensive')
print('%.2f%%\t\t%.2f%%' % (off, not_off))

Offensive	Not offensive
33.23%		66.77%


In [4]:
# Build vocab
id2word = {}
word2id = {}
i = 0 # index of unique word
for tweet in x_raw:
    for word in tweet.split():
        if word not in word2id:
            word2id[word] = i
            id2word[i] = word
            i += 1

# Build bag of words
data = []
rows = []
cols = []
for i in range(len(x_raw)):
    counts = {}
    for word in x_raw[i].split():
        if word2id[word] not in counts:
            counts[word2id[word]] = 1
        else:
            counts[word2id[word]] += 1
            
    for word_id, freq in counts.items():
        data.append(freq)
        rows.append(i)
        cols.append(word_id)
        
    # bias
    data.append(1)
    rows.append(i)
    cols.append(len(word2id))
        
x = csr_matrix((data, (rows, cols)))

In [5]:
# Shuffle x and y together
state = np.random.get_state()
i = np.arange(x.shape[0])
np.random.shuffle(i)
x =  x[i, :] # shuffling a sparse matrix is weird
np.random.set_state(state)
np.random.shuffle(y)

In [6]:
def sigmoid(x):
    s = np.zeros(x.shape)
    s[x > 0] = 1. / (1. + np.exp(-x[x > 0])) # avoid overflow
    s[x <= 0] = np.exp(x[x <= 0]) / (np.exp(x[x <= 0]) + 1) # avoid underflow
    return s

def cost(w, x, y):
    """ The cost function for logistic regression """
    h = sigmoid(x @ w)
    cost = np.average(-y * np.log(h) - (1 - y) * np.log(1 - h))
    return cost

def gradient_update(w, x, y):
    """ The gradient update for logistic regression"""
    h = sigmoid(x @ w)
    g = (h - y) @ x
    g = g / x.shape[0] 
    return g

def gradient_descent(w, x, y, alpha, iterations, print_iterations):
    """ Batch gradient descent algorithm """
    alpha *= x.shape[0]
    for i in range(iterations):
        if i % print_iterations == 0:
            print('%d) cost: %f' % (i, cost(w, x, y)))
        w -= alpha * gradient_update(w, x, y)       
    return w

def predict(w, x):
    """ Predict whether the label is 0 or 1 using learned logistic regression parameters """
    h = x @ w
    probabilities = sigmoid(h)
    predicted = 1 * (h > 0) # converts truth values to 1 or 0
    return probabilities, 1 * predicted

def report(w, x, metrics=['accuracy', 'precision', 'recall', 'auc']):
    """ Report metrics for `x` evaluated on the given LR classifier """
    # TODO
    prob, labels = predict(w, x)
    acc = labels[np.where(labels == y)].size / float(y.size) * 100
    pass

In [7]:
# k-fold cross validation
k = 5
chunk_size = x.shape[0] // k
alpha = 0.00002
iterations = 5000
print_iterations = 1000
total_acc = 0
for i in range(k):
    start = time()
    initial_w = np.random.rand(x.shape[1]) - 0.5
    x_train = vstack((x[:i * chunk_size], x[(i + 1) * chunk_size:]))
    y_train = np.concatenate((y[:i * chunk_size], y[(i + 1) * chunk_size:]))
    x_val = x[i * chunk_size:(i + 1) * chunk_size]
    y_val = y[i * chunk_size:(i + 1) * chunk_size]
    w = gradient_descent(initial_w.copy(), x_train, y_train, alpha, iterations, print_iterations)
    _, labels = predict(w, x_train)
    train_acc = labels[np.where(labels == y_train)].size / float(y_train.size) * 100
    print('train acc: %.4f%%' % train_acc)
    _, labels = predict(w, x_val)
    val_acc = labels[np.where(labels == y_val)].size / float(y_val.size) * 100
    total_acc += val_acc
    print('val acc: %.4f%%' % val_acc)
    print('took %.2fs' % (time() - start))
print('average validation accuracy: %.4f%%' % (total_acc / k))

0) cost: 0.879583
1000) cost: 0.597038
2000) cost: 0.536212
3000) cost: 0.495928
4000) cost: 0.465147
train acc: 80.3909%
val acc: 70.9215%
took 12.03s
0) cost: 1.399689
1000) cost: 0.598837
2000) cost: 0.537441
3000) cost: 0.496334
4000) cost: 0.464877
train acc: 80.2870%
val acc: 70.4683%
took 12.45s
0) cost: 0.924044
1000) cost: 0.598432
2000) cost: 0.536145
3000) cost: 0.494834
4000) cost: 0.463548
train acc: 80.5230%
val acc: 72.5453%
took 12.31s
0) cost: 0.989976
1000) cost: 0.591985
2000) cost: 0.528353
3000) cost: 0.487456
4000) cost: 0.456792
train acc: 81.1745%
val acc: 71.2613%
took 12.22s
0) cost: 0.821206
1000) cost: 0.587970
2000) cost: 0.528595
3000) cost: 0.488567
4000) cost: 0.457988
train acc: 81.1934%
val acc: 70.6571%
took 16.87s
average validation accuracy: 71.1707%
